# Investigating SFPD incident reports

### By Eric Sagara

This walkthrough is designed to show you how I approach a new data analysis. It requires a couple of libraries that you may or may not be familiar with. The first is [Pandas](https://pandas.pydata.org/docs/) which is installed by using `pip install pandas`. The second is numpy, which is a Pandas dependency - in other words it should automatically install when you install pandas. Lastly is the charting library [Altair](https://altair-viz.github.io/) and it's dependency Vega. These are installed using `pip install altair vega`.

Let's begin by importing our libraries. In Jupyter Notebook you need to enable Altair's rendering engine by adding `alt.renderers.enable('notebook')` after importing.

In [1]:
import pandas as pd
import numpy as np
import altair as alt

## Loading data

Before we load the data we should take a look at the [documentation](https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-2018-to-Present/wg3w-h783). 

There are two methods of loading the data. You can either download the data ahead of time and load the file or just pass the url where the file is located into pandas' `read_csv` method. We are also going to parse several of the date fields into python `datetime` objects on import. This will allow us to do math with dates among other things.

In [2]:
# Use the filepath url if the data is stored on your computer, 
# otherwise use the csv_url to download and load it into pandas
incidents_filepath = 'data/sfpd_incidents_2018_to_present_20190228.csv'
population_filepath = 'data/SF_neighborhood_pop.csv'
# csv_url = 'https://data.sfgov.org/api/views/wg3w-h783/rows.csv?accessType=DOWNLOAD'

# Note we use the parse_dates parameter to convert the 
# Incident Date field values to datetime objects
incidents = pd.read_csv(incidents_filepath, parse_dates = ['Incident Date','Incident Datetime','Report Datetime'])

## First look at the data

Let's start to get a general idea of what the data looks like. We begin simply by lookiing at the first few rows of the data.

In [3]:
incidents.head()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,Incident Description,Resolution,Intersection,CNN,Police District,Analysis Neighborhood,Supervisor District,Latitude,Longitude,point
0,2018-10-22 10:50:00,2018-10-22,10:50,2018,Monday,2018-10-22 11:24:00,72897505073,728975,180801798,182951518.0,...,"Burglary, Other Bldg., Unlawful Entry",Open or Active,VAN NESS AVE \ OAK ST,30705000.0,Northern,Tenderloin,6.0,37.775422,-122.419397,"(37.7754222198843, -122.41939688941916)"
1,2018-08-24 14:50:00,2018-08-24,14:50,2018,Friday,2018-08-24 14:52:00,72925174010,729251,180638466,181362475.0,...,Missing Juvenile,Open or Active,28TH AVE \ VICENTE ST,23250000.0,Taraval,Sunset/Parkside,4.0,37.738900,-122.485074,"(37.738899869536326, -122.48507449528636)"
2,2018-10-14 14:54:00,2018-10-14,14:54,2018,Sunday,2018-10-24 13:10:00,72980906303,729809,180807520,182971760.0,...,"Theft, From Building, $200-$950",Open or Active,PINTO AVE \ ARBALLO DR,23104000.0,Taraval,Lakeshore,7.0,37.720450,-122.483015,"(37.7204499957522, -122.48301534194144)"
3,2018-10-14 14:54:00,2018-10-14,14:54,2018,Sunday,2018-10-24 13:10:00,72980964070,729809,180807520,182971760.0,...,Suspicious Occurrence,Open or Active,PINTO AVE \ ARBALLO DR,23104000.0,Taraval,Lakeshore,7.0,37.720450,-122.483015,"(37.7204499957522, -122.48301534194144)"
4,2018-10-18 16:40:00,2018-10-18,16:40,2018,Thursday,2018-10-26 07:42:00,73011316650,730113,180791014,182911410.0,...,Methamphetamine Offense,Cite or Arrest Adult,BOARDMAN PL \ BRYANT ST,23914000.0,Southern,South of Market,6.0,37.775161,-122.403636,"(37.7751608100771, -122.40363551943442)"


Next let's get a deeper sense of what the individual columns are.

In [4]:
incidents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176242 entries, 0 to 176241
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   Incident Datetime        176242 non-null  datetime64[ns]
 1   Incident Date            176242 non-null  datetime64[ns]
 2   Incident Time            176242 non-null  object        
 3   Incident Year            176242 non-null  int64         
 4   Incident Day of Week     176242 non-null  object        
 5   Report Datetime          176242 non-null  datetime64[ns]
 6   Row ID                   176242 non-null  int64         
 7   Incident ID              176242 non-null  int64         
 8   Incident Number          176242 non-null  int64         
 9   CAD Number               135931 non-null  float64       
 10  Report Type Code         176242 non-null  object        
 11  Report Type Description  176242 non-null  object        
 12  Filed Online    

### Checking to see how complete the data is

Often there is a lag in data collection or early data collection efforts could be spotty. This means we could have days, months or even years with incomplete data. We should check this to make sure we are working with a complete data set. We can do this using the `Incident Date` column.

In [5]:
incidents['Incident Date'].describe(datetime_is_numeric=True)

count                           176242
mean     2018-07-26 04:37:17.596032768
min                2018-01-01 00:00:00
25%                2018-04-14 00:00:00
50%                2018-07-26 00:00:00
75%                2018-11-04 00:00:00
max                2019-02-27 00:00:00
Name: Incident Date, dtype: object

This tells us data collection began on January 1, 2018 and the data is current through February 27, 2019. However that doesn't give us the entire picture. Rather than examine the data day-by-day, let's start by doing a `groupby` operation using the `Incident Date` and the `Row ID` column that has unique values.

In [6]:
incidents_by_day = incidents[['Incident Date', 'Row ID']].groupby('Incident Date', as_index = False).count()
incidents_by_day.head()

,Incident Date,Row ID
0,2018-01-01,483
1,2018-01-02,399
2,2018-01-03,431
3,2018-01-04,472
4,2018-01-05,466


And let's use Altair to dive in further, but first let's rename the `Row ID` column into something that makes sense.

We start by creating a `Chart` object and passing in our data that we want to chart. Then we specify what type of chart we want - in this case a line chart - and finally what the X and Y axes will be.

In [7]:
incidents_by_day.rename(columns = {'Row ID' : 'Crimes'}, inplace = True)

alt.Chart(incidents_by_day).mark_line().encode(x = 'Incident Date', y = 'Crimes')

alt.Chart(...)

It's hard to see, but it looks like there is a dip in reported crimes near the end of our data collection period. We can get a closer look by creating a "zoomed in" look at the data. We are going to create the same chart as above, but this time filter for only those crimes reported in 2019 using the `dt.year` method to return only the year of the datetime object.

In [8]:
alt.Chart(
    incidents_by_day[incidents_by_day['Incident Date'].dt.year == 2019]
    ).mark_line(
    ).encode(
        x = 'Incident Date', 
        y = 'Crimes'
    )

alt.Chart(...)

It's immediately clear that there is a lag in data collection for the last few days in February. We should probably correct for that by excluding at least February. In this case we are going to filter our data so we are only dealing with 2018 crimes.

In [9]:
incidents = incidents[incidents['Incident Date'].dt.year == 2018]

### Looking for unique IDs

There are several columns which appear to possibly be unique IDs for incidents. We should check and make sure they are what we think they are.

In [10]:
id_columns = ['Row ID', 'Incident ID', 'Incident Number', 'CAD Number']

for column in id_columns:
    print(column, incidents[column].duplicated().unique())

Row ID [False]
Incident ID [False  True]
Incident Number [False  True]
CAD Number [False  True]


This shows us that there are duplicate values in every column except `Row ID`. This is important to keep in mind. It means there are multiple incidents or crimes associated with a given report. Why is this important? Well many times we like to conduct sanity checks by comparing total numbers of crimes against other publically available data sources. Other times we want to compare one city's crime rates against anothers. To do so you will need to access another data source such as the FBI's UCR program.

There's a large caveat when working with UCR data. The FBI requires that each report be filed only once under a single crime category. The crime category is based off of the most serious offense and any other crimes associated with a given incident would not appear in the FBI data.

For example, let's say someone was kidnapped and then murdered. In the SFPD data this would appear as two records - one for the kidnapping and another for the murder. However, in the FBI data this would only be counted as a murder, the kidnapping would be ignored for the purposes of data collection.

This means that any numbers generated from SFPD most likely will not match up exactly with FBI figures. This is not necessarily a big deal. You are not looking for exact matches when conducting sanity checks between multiple sources, you just want to make sure your numbers are in the ballpark with other publically available figures.

## Dealing with duplicates

We know from the documentation that the `Incident ID` and `Incident Number` fields are the unique identifiers. Our work above shows us that there are duplicate records. One cause could be multiple crimes per incident. Another could be supplemental reports. We need to account for both in our analysis.

In [11]:
incidents['Report Type Description'].value_counts()

Initial                97353
Coplogic Initial       29964
Initial Supplement     13091
Vehicle Initial         6485
Vehicle Supplement      4609
Coplogic Supplement     3757
Name: Report Type Description, dtype: int64

For the purposes we are going to exclude those reports filed online - `Coplogic Initial` and `Coplogic Supplement`. We also don't know what `Vehicle Initial` and `Vehicle Supplement` so pending further reporting we are going to omit those as well.

In [12]:
reports_to_keep = [
    'Initial',
    'Initial Supplement'
]
incidents = incidents[incidents['Report Type Description'].isin(reports_to_keep)]

Next we are going to sort the dataframe by the `Report Datetime` column. By default the `sort_values` function sorts the data in ascending order, which is the behavior we want.

In [13]:
incidents.sort_values(by='Report Datetime', inplace=True)

The two fields that make a crime and incident unique appear to be the `Incident ID` and the `Incident Code` fields according to the documentation. The `Incident ID` is the unique identifier for an incident report. The `Incident Code` is the identifier for the type of incident. Together the two make a unique record. Now we can find duplicate records (cases where a supplemental report was filed) using those two columns and drop them from the data set. We want to keep the last duplicate record found since we sorted the data by `Report Datetime` in ascending order. This ensures that we get the most current `Resolution`.

In [14]:
incidents.drop_duplicates(subset=['Incident ID','Incident Code'], keep='last', inplace=True)

The `Resolution` column which indicates the status of the crime at the time the report was created. However, we can use this to create a `case_status` column to show the status of the case at the time of the filing. Let's start by seeing what the unique value in the `Resolution` field are.

In [15]:
incidents['Resolution'].unique()

array(['Cite or Arrest Adult', 'Open or Active',
       'Cite or Arrest Juvenile', 'Exceptional Adult', 'Unfounded',
       'Exceptional Juvenile'], dtype=object)

Uhoh. One of the possible values for the `Resolution` column is `Unfounded` which means the responding officer found no crime at the incident. You may find value in this depending on what the goal of your analysis is, but in this case we should probably filter that out of our dataset.

In [16]:
incidents = incidents[incidents['Resolution'] != 'Unfounded']

Ok now that we have done that we can start coding our `case_status` column. We will take advantage of Pandas' `.loc` function to target specific columns and rows in the data set.

The `.loc` function works in this case by passing in what we want to filter rows by along with which column we want to alter. First let's create the column and assign null values to it using numpy's `nan`.

In [17]:
incidents['case_status'] = np.nan
incidents['case_status'].head()

62668   NaN
62669   NaN
62670   NaN
62667   NaN
61449   NaN
Name: case_status, dtype: float64

Now let's code in open vs closed cases.

In [18]:
incidents.loc[incidents['Resolution'] == 'Open or Active', 'case_status'] = 'Open'
incidents['case_status'].head()

62668     NaN
62669     NaN
62670     NaN
62667     NaN
61449    Open
Name: case_status, dtype: object

We're halfway there - we still have `NaN` values for cases that were closed. Rather than use the `.loc` method we can take a simpler approach by replacing `NaN` values with `closed`

In [19]:
incidents['case_status'].fillna('Closed', inplace = True)
incidents['case_status'].head()

62668    Closed
62669    Closed
62670    Closed
62667    Closed
61449      Open
Name: case_status, dtype: object

## Digging in with some analysis

Let's start by finding out what crimes are reported in this data and which ones are the most common.

In [20]:
incidents['Incident Category'].unique()

array(['Vehicle Impounded', 'Traffic Violation Arrest',
       'Other Miscellaneous', 'Weapons Carrying Etc', 'Non-Criminal',
       'Larceny Theft', 'Assault',
       'Offences Against The Family And Children', 'Fire Report',
       'Robbery', 'Malicious Mischief', 'Disorderly Conduct', 'Warrant',
       'Miscellaneous Investigation', 'Burglary', 'Suspicious Occ',
       'Civil Sidewalks', 'Drug Offense', 'Missing Person', 'Fraud',
       'Weapons Offense', 'Motor Vehicle Theft', 'Stolen Property',
       'Lost Property', 'Other Offenses', 'Traffic Collision', 'Suicide',
       'Other', 'Homicide', 'Family Offense',
       'Forgery And Counterfeiting', 'Case Closure', 'Sex Offense',
       'Arson', 'Courtesy Report', 'Gambling', 'Recovered Vehicle',
       'Drug Violation', 'Prostitution', 'Juvenile Offenses',
       'Embezzlement', 'Vandalism', nan,
       'Human Trafficking (A), Commercial Sex Acts', 'Liquor Laws',
       'Rape', 'Suspicious', 'Motor Vehicle Theft?', 'Weapons Offenc

Oops. There are `NaN` values in this column as well. Let's get an idea of how often that occurs.

In [21]:
incidents.loc[incidents['Incident Category'].isna() == True, 'Row ID'].count()

15

Ok good, there are only 7 crimes where this is true. We could filter this out but by default Pandas should exclude these from most of the functions we use to analyze the data. Let's get an idea of the ten most common crimes.

In [22]:
crime_count = incidents[['Incident Category', 'Row ID']].groupby('Incident Category', as_index = False).count()
crime_count.rename(columns = {'Row ID' : 'Crimes'}, inplace = True)

crime_count.sort_values('Crimes', ascending = False).head(10)

,Incident Category,Crimes
19,Larceny Theft,21275
30,Other Miscellaneous,11889
27,Non-Criminal,9473
1,Assault,9088
2,Burglary,6477
22,Malicious Mischief,6068
46,Warrant,5626
24,Missing Person,4430
13,Fraud,4145
7,Drug Offense,4101


We can also check to see how often crimes are closed at the time the report was filed using a pivot table. While we are at it, let's calculate each incident category's percentage of all crimes.

In [23]:
status_by_crime = incidents.pivot_table(
    index = 'Incident Category', 
    columns = 'case_status', 
    values = 'Row ID', 
    aggfunc = 'count', 
    margins = True, 
    margins_name = "Total"
)

status_by_crime = pd.DataFrame(status_by_crime.to_records())
status_by_crime

,Incident Category,Closed,Open,Total
0,Arson,93.0,262.0,355
1,Assault,3044.0,6044.0,9088
2,Burglary,1086.0,5391.0,6477
3,Case Closure,382.0,13.0,395
4,Civil Sidewalks,375.0,126.0,501
5,Courtesy Report,27.0,367.0,394
6,Disorderly Conduct,883.0,1645.0,2528
7,Drug Offense,3794.0,307.0,4101
8,Drug Violation,54.0,1.0,55
9,Embezzlement,26.0,133.0,159


We now have some aggregated information about crimes. We also have a new column and a new row, both named `Total`. The row `Total` is useful, but we still want to strip it out after saving the total number of crimes figure.

In [24]:
total_crimes = status_by_crime.loc[status_by_crime['Incident Category'] == 'Total', 'Total'].values[0].copy()

# drop the total row
status_by_crime.drop(status_by_crime[status_by_crime['Incident Category'] == 'Total'].index, inplace=True)

Let's calculate some rates.

In [25]:
# First a `closure_rate`

status_by_crime['closure_rate'] = (status_by_crime['Closed'] / status_by_crime['Total']) * 100.0

# And then a percent of total crimes

status_by_crime['percent_of_all_crimes'] = (status_by_crime['Total'] / total_crimes) * 100.0

status_by_crime.head()

,Incident Category,Closed,Open,Total,closure_rate,percent_of_all_crimes
0,Arson,93.0,262.0,355,26.197183,0.324773
1,Assault,3044.0,6044.0,9088,33.494718,8.314198
2,Burglary,1086.0,5391.0,6477,16.767022,5.925513
3,Case Closure,382.0,13.0,395,96.708861,0.361368
4,Civil Sidewalks,375.0,126.0,501,74.850299,0.458342


Let's try some charting in Altair and make a scatter plot comparing closure rates to crime percentage.

In [26]:
alt.Chart(
    status_by_crime
).mark_point(
).encode(
    x = 'percent_of_all_crimes',
    y = 'closure_rate',
    tooltip=['Incident Category', 'percent_of_all_crimes', 'closure_rate'],
)

alt.Chart(...)

This type of chart is known as a scatterplot. It is used to check for a relationship or correlation between two different variables - in this case `percent_of_all_crimes` and `closure_rate`. There is very little clustering of the data points here, indicating that there is no real relationship. So there is not a lot to glean from this chart, other than the fact that the most common crime has a very low closure rate. So what crime is that?

In [27]:
status_by_crime.sort_values('percent_of_all_crimes', ascending = False).head(1)

,Incident Category,Closed,Open,Total,closure_rate,percent_of_all_crimes
19,Larceny Theft,2279.0,18996.0,21275,10.712103,19.463529


So larcenies are the most common crime. The low closure rate at the time the report was filed is not surprising either. A lot of times an officer doesn't even respond to these types of calls. Instead victims fill out police reports and submit them online or by mail. But it warrants looking further into larcenies.

## Diving deeper into larcenies

Let's start by isolating those records related to larceny and then look at the subcategories for larceny.

In [28]:
larcenies = incidents.loc[incidents['Incident Category'] == 'Larceny Theft',:].copy()
larcenies.head()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,...,Resolution,Intersection,CNN,Police District,Analysis Neighborhood,Supervisor District,Latitude,Longitude,point,case_status
62666,2018-01-01 00:40:00,2018-01-01,00:40,2018,Monday,2018-01-01 01:40:00,61869706113,618697,180000207,180010493.0,...,Open or Active,GEARY ST \ POWELL ST,24903000.0,Central,Financial District/South Beach,3.0,37.787359,-122.408227,"(37.78735926098589, -122.40822672700406)",Open
62677,2018-01-01 02:08:00,2018-01-01,02:08,2018,Monday,2018-01-01 02:08:00,61871005015,618710,180000285,180010537.0,...,Cite or Arrest Adult,CESAR CHAVEZ ST \ CAPP ST \ MISSION ST,21304000.0,Mission,Bernal Heights,9.0,37.748166,-122.418221,"(37.74816568813204, -122.41822117169174)",Closed
62687,2018-01-01 03:37:00,2018-01-01,03:37,2018,Monday,2018-01-01 03:37:00,61872206153,618722,180000451,180010843.0,...,Open or Active,CLARA ST \ 05TH ST,23910000.0,Southern,South of Market,6.0,37.779459,-122.402377,"(37.779458714779906, -122.40237749311285)",Open
62708,2018-01-01 06:55:00,2018-01-01,06:55,2018,Monday,2018-01-01 07:13:00,61892706302,618927,180000649,180011062.0,...,Open or Active,ARLETA AVE \ BAY SHORE BLVD \ SAN BRUNO AVE,20298000.0,Ingleside,Visitacion Valley,10.0,37.712179,-122.402459,"(37.71217948507751, -122.40245948022154)",Open
62720,2018-01-01 08:17:00,2018-01-01,08:17,2018,Monday,2018-01-01 08:28:00,61894406244,618944,180000752,180011308.0,...,Open or Active,ELLIS ST \ WEBSTER ST,26585000.0,Northern,Western Addition,5.0,37.782570,-122.430798,"(37.782569887796356, -122.43079788213421)",Open


In [29]:
larcenies[['Incident Subcategory', 'Row ID']].groupby('Incident Subcategory', as_index = False).count()

,Incident Subcategory,Row ID
0,Larceny - Auto Parts,73
1,Larceny - From Vehicle,10179
2,Larceny Theft - Bicycle,482
3,Larceny Theft - From Building,3020
4,Larceny Theft - Other,3610
5,Larceny Theft - Pickpocket,794
6,Larceny Theft - Purse Snatch,30
7,Larceny Theft - Shoplifting,2693
8,Theft From Vehicle,394


Wait a minute. There are two subcategories that appear to be the same - `Larceny - From Vehicle` and `Theft From Vehicle`. Let's combine those two and re-run this portion of the analysis.

In [30]:
larcenies['Incident Subcategory'].replace({'Theft From Vehicle' : 'Larceny - From Vehicle'}, inplace = True)

larcenies[['Incident Subcategory', 'Row ID']].groupby('Incident Subcategory', as_index = False).count()

,Incident Subcategory,Row ID
0,Larceny - Auto Parts,73
1,Larceny - From Vehicle,10573
2,Larceny Theft - Bicycle,482
3,Larceny Theft - From Building,3020
4,Larceny Theft - Other,3610
5,Larceny Theft - Pickpocket,794
6,Larceny Theft - Purse Snatch,30
7,Larceny Theft - Shoplifting,2693


So theft from vehicles is the most common type of larceny. Let's isolate that and dig in further.

In [31]:
thefts_from_car = larcenies.loc[larcenies['Incident Subcategory'] == 'Larceny - From Vehicle', :]
thefts_from_car['Row ID'].count()

10573

How often are cases closed at the time of when the report was filed?

In [32]:
thefts_status = thefts_from_car['case_status'].value_counts()
thefts_status

Open      9989
Closed     584
Name: case_status, dtype: int64

In [33]:
(thefts_status['Closed'] / (thefts_status['Open'] + thefts_status['Closed'])) * 100.0

5.523503263028469

A little more than 5 percent of the time. That's not very high. Let's ask some other questions.

### Where are these types of thefts most common?

In [34]:
by_neighborhood = thefts_from_car[['Analysis Neighborhood', 'Row ID']].groupby(
    'Analysis Neighborhood', 
    as_index = False
).count()

by_neighborhood.rename(columns = {'Row ID' : 'Thefts'}, inplace = True)

#by_neighborhood = by_district.sort_values('Thefts', ascending = False)
by_neighborhood = by_neighborhood.sort_values('Thefts', ascending = False)

by_neighborhood.head(10)

,Analysis Neighborhood,Thefts
18,Mission,978
22,North Beach,973
31,Russian Hill,875
5,Financial District/South Beach,787
33,South of Market,650
9,Hayes Valley,443
40,Western Addition,422
35,Tenderloin,419
25,Outer Richmond,348
0,Bayview Hunters Point,331


In [35]:
alt.Chart(by_neighborhood).mark_bar().encode(x = 'Thefts', y = alt.Y('Analysis Neighborhood', sort='-x'))

alt.Chart(...)

### Correcting for population
This analysis isn't entirely accurate, we should probably correct for population. We start by loading in our population data set.

In [36]:
population_data = pd.read_csv(population_filepath)
population_data.head()

,Neighborhood,Population,Margin of Error
0,Bayview Hunters Point,37394,4462
1,Bernal Heights,25858,2146
2,Castro/Upper Market,22284,1498
3,Chinatown,14737,1575
4,Excelsior,40701,3689


Next we join the two data sets together using the `merge` function.

In [37]:
neighborhoods = by_neighborhood.merge(population_data, left_on='Analysis Neighborhood', right_on='Neighborhood', how='left')
neighborhoods.drop(columns='Neighborhood', inplace=True)
neighborhoods.head()

,Analysis Neighborhood,Thefts,Population,Margin of Error
0,Mission,978,59639,5335
1,North Beach,973,11636,1140
2,Russian Hill,875,17830,1584
3,Financial District/South Beach,787,19458,2200
4,South of Market,650,21771,2115


In [38]:
neighborhoods['crime_rate'] = (neighborhoods['Thefts'] / neighborhoods['Population']) * 10000
neighborhoods.sort_values(by='crime_rate', inplace=True, ascending=False)
neighborhoods

,Analysis Neighborhood,Thefts,Population,Margin of Error,crime_rate
11,Golden Gate Park,284,66,36,43030.303030
37,Lincoln Park,43,305,103,1409.836066
1,North Beach,973,11636,1140,836.198006
15,Japantown,235,3532,393,665.345413
2,Russian Hill,875,17830,1584,490.745934
3,Financial District/South Beach,787,19458,2200,404.460890
4,South of Market,650,21771,2115,298.562308
5,Hayes Valley,443,19678,1586,225.124505
19,Twin Peaks,177,8019,685,220.725776
6,Western Addition,422,22638,2044,186.412227


In [39]:
alt.Chart(neighborhoods).mark_bar().encode(x = 'crime_rate', y = alt.Y('Analysis Neighborhood', sort='-x'))

alt.Chart(...)

In [40]:

alt.Chart(
    neighborhoods[neighborhoods['Population'] >= 1000]
).mark_bar().encode(
    x = 'crime_rate',
    y = alt.Y(
        'Analysis Neighborhood',
        sort='-x'
    )
)

alt.Chart(...)

### What time of day are thefts from cars most common?

In [41]:
by_hour = thefts_from_car[['Incident Datetime', 'Row ID']].groupby(thefts_from_car['Incident Datetime'].dt.hour.rename('Hour')).agg({'Row ID': 'count'})
by_hour = pd.DataFrame(by_hour.to_records())
by_hour.rename(columns = {'Row ID' : 'Thefts'}, inplace = True)

alt.Chart(by_hour).mark_bar().encode(y = 'Thefts', x = 'Hour:O')

alt.Chart(...)